In [1]:
# coding: utf-8

# In[1]:
from PyQt5.QtCore import Qt, QEvent
from PyQt5.QtGui import QImage, QPixmap, QPalette, QPainter
from PyQt5.QtPrintSupport import QPrintDialog, QPrinter
from PyQt5.QtWidgets import QLabel, QSizePolicy, QScrollArea, QMessageBox, QMainWindow, QMenu, QAction, qApp, QFileDialog
import os
import qimage2ndarray
import cv2

class QImageViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.image_dir = "/home/hoseung/Dropbox/DeepInsight/2021ETRI/BBS_data/Preds/a/c4_plot/overlap/"
        self.image_list = os.listdir(self.image_dir)
        self.image_list.sort()
        #print(self.image_list)

        self.printer = QPrinter()
        self.scaleFactor = 0.5

        self.imageLabel = QLabel()
        self.imageLabel.setBackgroundRole(QPalette.Base)
        #self.imageLabel.setSizePolicy(QSizePolicy.Ignored, QSizePolicy.Ignored)
        self.imageLabel.setScaledContents(True)

        self.scrollArea = QScrollArea()
        self.scrollArea.setBackgroundRole(QPalette.Dark)
        self.scrollArea.setWidget(self.imageLabel)
        self.scrollArea.setVisible(False)

        self.setCentralWidget(self.scrollArea)

        self.createActions()
        self.createMenus()

        #Mouse
        self.setMouseTracking(True)

        self.cnt = 0
        self.fout = open("output.txt", "w")


        self.setWindowTitle("Image Viewer")
        self.resize(1600, 1600)

    def open(self):
        options = QFileDialog.Options()
        # fileName = QFileDialog.getOpenFileName(self, "Open File", QDir.currentPath())
        dir_root = str(QFileDialog.getExistingDirectory(self, "Select Directory" ))
        #print(dir_root)
        image_list = os.listdir(dir_root)
        image_list.sort()
        #print(image_list)
        #print(dir_root)
        fileName = os.path.join(dir_root, image_list[0])
        if fileName:
            image = QImage(fileName)
            if image.isNull():
                QMessageBox.information(self, "Image Viewer", "Cannot load %s." % fileName)
                return

            self.imageLabel.setPixmap(QPixmap.fromImage(image))
            self.scaleFactor = 1.0

            self.scrollArea.setVisible(True)
            self.printAct.setEnabled(True)
            self.fitToWindowAct.setEnabled(True)
            self.updateActions()

            if not self.fitToWindowAct.isChecked():
                self.imageLabel.adjustSize()

    def mousePressEvent(self, e):
        print(f'image len : {len(self.image_list)}')
        print(f'cnt {self.cnt}')
        if self.cnt == len(self.image_list):
            self.fout.close()
            self.close()
        elif e.buttons() & Qt.LeftButton:
            x = e.x()
            y = e.y()
            print(f'x : {x} y : {y}')

            self.fout.write(f'{self.image_list[self.cnt]} {x} {y} \n')
            self.fout.flush()
            self.next_image()
            self.cnt +=1

    def next_image(self):
        fileName = os.path.join(self.image_dir, self.image_list[self.cnt])
        if fileName:
            image = cv2.imread(fileName)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, dsize=(1300,1300))
            image = QPixmap(self.realsenseFrameToQImage(image))
            if image.isNull():
                QMessageBox.information(self, "Image Viewer", "Cannot load %s." % fileName)
                return

            self.imageLabel.setPixmap(image)

            self.scrollArea.setVisible(True)
            self.printAct.setEnabled(True)
            self.fitToWindowAct.setEnabled(True)
            self.updateActions()

            if not self.fitToWindowAct.isChecked():
                self.imageLabel.adjustSize()

    def realsenseFrameToQImage(self, frame):
        result = qimage2ndarray.array2qimage(frame)
        return result

    def print_(self):
        dialog = QPrintDialog(self.printer, self)
        if dialog.exec_():
            painter = QPainter(self.printer)
            rect = painter.viewport()
            size = self.imageLabel.pixmap().size()
            size.scale(rect.size(), Qt.KeepAspectRatio)
            painter.setViewport(rect.x(), rect.y(), size.width(), size.height())
            painter.setWindow(self.imageLabel.pixmap().rect())
            painter.drawPixmap(0, 0, self.imageLabel.pixmap())
    
    def eventFilter(self, source, event):
        if event.type() == QtCore.QEvent.MouseButtonPress:
            if event.button() == QtCore.Qt.LeftButton:
                print(event.pos())
                

    def zoomIn(self):
        self.scaleImage(1.25)

    def zoomOut(self):
        self.scaleImage(0.8)

    def normalSize(self):
        self.imageLabel.adjustSize()
        self.scaleFactor = 1.0

    def fitToWindow(self):
        fitToWindow = self.fitToWindowAct.isChecked()
        self.scrollArea.setWidgetResizable(fitToWindow)
        if not fitToWindow:
            self.normalSize()

        self.updateActions()

    def createActions(self):
        self.openAct = QAction("&Open...", self, shortcut="Ctrl+O", triggered=self.open)
        self.printAct = QAction("&Print...", self, shortcut="Ctrl+P", enabled=False, triggered=self.print_)
        self.exitAct = QAction("E&xit", self, shortcut="Ctrl+Q", triggered=self.close)
        self.zoomInAct = QAction("Zoom &In (25%)", self, shortcut="Ctrl++", enabled=False, triggered=self.zoomIn)
        self.zoomOutAct = QAction("Zoom &Out (25%)", self, shortcut="Ctrl+-", enabled=False, triggered=self.zoomOut)
        self.normalSizeAct = QAction("&Normal Size", self, shortcut="Ctrl+S", enabled=False, triggered=self.normalSize)
        self.fitToWindowAct = QAction("&Fit to Window", self, enabled=False, checkable=True, shortcut="Ctrl+F",
                                      triggered=self.fitToWindow)

    def createMenus(self):
        self.fileMenu = QMenu("&File", self)
        self.fileMenu.addAction(self.openAct)
        self.fileMenu.addAction(self.printAct)
        self.fileMenu.addSeparator()
        self.fileMenu.addAction(self.exitAct)

        self.viewMenu = QMenu("&View", self)
        self.viewMenu.addAction(self.zoomInAct)
        self.viewMenu.addAction(self.zoomOutAct)
        self.viewMenu.addAction(self.normalSizeAct)
        self.viewMenu.addSeparator()
        self.viewMenu.addAction(self.fitToWindowAct)

        self.menuBar().addMenu(self.fileMenu)
        self.menuBar().addMenu(self.viewMenu)

    def updateActions(self):
        self.zoomInAct.setEnabled(not self.fitToWindowAct.isChecked())
        self.zoomOutAct.setEnabled(not self.fitToWindowAct.isChecked())
        self.normalSizeAct.setEnabled(not self.fitToWindowAct.isChecked())

    def scaleImage(self, factor):
        self.scaleFactor *= factor
        self.imageLabel.resize(self.scaleFactor * self.imageLabel.pixmap().size())

        self.adjustScrollBar(self.scrollArea.horizontalScrollBar(), factor)
        self.adjustScrollBar(self.scrollArea.verticalScrollBar(), factor)

        self.zoomInAct.setEnabled(self.scaleFactor < 3.0)
        self.zoomOutAct.setEnabled(self.scaleFactor > 0.333)

    def adjustScrollBar(self, scrollBar, factor):
        scrollBar.setValue(int(factor * scrollBar.value()
                               + ((factor - 1) * scrollBar.pageStep() / 2)))


if __name__ == '__main__':
    import sys
    from PyQt5.QtWidgets import QApplication

    app = QApplication(sys.argv)
    imageViewer = QImageViewer()
    imageViewer.show()
    sys.exit(app.exec_())
    # TODO QScrollArea support mouse
    # base on https://github.com/baoboa/pyqt5/blob/master/examples/widgets/imageviewer.py
    #
    # if you need Two Image Synchronous Scrolling in the window by PyQt5 and Python 3
    # please visit https://gist.github.com/acbetter/e7d0c600fdc0865f4b0ee05a17b858f2


image len : 220
cnt 0
x : 841 y : 370
image len : 220
cnt 1
x : 586 y : 394
image len : 220
cnt 2
x : 573 y : 353
image len : 220
cnt 3
x : 150 y : 314
image len : 220
cnt 4
x : 596 y : 1199
image len : 220
cnt 5
x : 548 y : 979
image len : 220
cnt 6
x : 171 y : 147
image len : 220
cnt 7
x : 141 y : 999
image len : 220
cnt 8
x : 600 y : 320
image len : 220
cnt 9
x : 590 y : 1208
image len : 220
cnt 10
x : 546 y : 1199
image len : 220
cnt 11
x : 555 y : 1216
image len : 220
cnt 12
x : 161 y : 541
image len : 220
cnt 13
x : 161 y : 541
image len : 220
cnt 14
x : 774 y : 370
image len : 220
cnt 15
x : 754 y : 347
image len : 220
cnt 16
x : 597 y : 347
image len : 220
cnt 17
x : 561 y : 1167
image len : 220
cnt 18
x : 560 y : 124
image len : 220
cnt 19
x : 396 y : 150
image len : 220
cnt 20
x : 594 y : 1193
image len : 220
cnt 21
x : 170 y : 993
image len : 220
cnt 22
x : 537 y : 1214
image len : 220
cnt 23
x : 773 y : 347
image len : 220
cnt 24
x : 764 y : 371
image len : 220
cnt 25
x : 7

image len : 220
cnt 207
x : 328 y : 173
image len : 220
cnt 208
x : 776 y : 751
image len : 220
cnt 209
x : 770 y : 764
image len : 220
cnt 210
x : 770 y : 764
image len : 220
cnt 211
x : 130 y : 785
image len : 220
cnt 212
x : 548 y : 1187
image len : 220
cnt 213
x : 1025 y : 548
image len : 220
cnt 214
x : 528 y : 780
image len : 220
cnt 215
x : 783 y : 813
image len : 220
cnt 216
x : 766 y : 793
image len : 220
cnt 217
x : 579 y : 1182
image len : 220
cnt 218
x : 781 y : 770
image len : 220
cnt 219
x : 781 y : 770
image len : 220
cnt 220


SystemExit: 0

/home/hoseung/anaconda3/envs/deepinsight/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
